In [249]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
%run ../lib_data_operations.py

In [250]:
(df_etf_init, df_orders_init, df_incomes, df_prices_init, _) = load_data()
(df_orders, _) = preprocess_orders(df_orders_init)
df_prices = preprocess_prices(df_prices_init)

df_etf = preprocess_etf_masterdata(df_etf_init)
orders_etf = enrich_orders(df_orders, df_etf)

In [261]:
str(get_portfolio_value(orders_etf, df_prices)["last_price_update"].iloc[0]).split(" ")[0]

'2021-01-19'

In [255]:
df_timeseries = prepare_timeseries(df_orders)

In [248]:
df_orders

,Index,Date,Price,Investment,Ordercost,Depotprovider,Name,ISIN,Amount
0,0,2020-02-01,54.2143,50.0,-0.00,flatex,Xtrackers EURO STOXX 50 UCITS ETF 1C,LU0380865021,0.922266
1,0,2020-02-01,30.9100,50.0,-0.00,traderepublic,iShares BRIC 50 UCITS ETF,IE00B1W57M07,1.617599
2,0,2020-02-01,27.7100,25.0,-0.00,traderepublic,iShares Core MSCI Emerging Markets IMI UCITS E...,IE00BKM4GZ66,0.902201
3,0,2020-02-01,6.9200,25.0,-0.00,traderepublic,iShares Edge MSCI USA Value Factor UCITS ETF,IE00BD1F4M44,3.612717
4,0,2020-02-01,311.3800,25.0,-0.00,traderepublic,iShares Core S&P 500 UCITS ETF (Acc),IE00B5BMR087,0.080288
...,...,...,...,...,...,...,...,...,...
140,11,2021-01-01,50.9400,25.0,0.37,comdirect,EUWAX Gold,DE000EWG0LD1,0.490773
141,11,2021-01-01,33.8600,25.0,-0.00,comdirect,VanEck Vectors Global Real Estate UCITS ETF,NL0009690239,0.738334
142,11,2021-01-01,116.7800,50.0,-0.00,comdirect,Amundi Nasdaq 100 UCITS ETF EUR (C),LU1681038243,0.428156
143,11,2021-01-01,92.4400,25.0,-0.00,comdirect,ComStage Commerzbank Commodity ex-Agriculture ...,LU0419741177,0.270446


In [247]:
stockname = df_orders["Name"].iloc[3]
stockname = "Overall Portfolio"
df_timeseries[df_timeseries["Name"] == stockname]

,Date,Name,Investment,Ordercost,Value
204,2020-02-01,Overall Portfolio,175.0,0.000,175.000000
205,2020-03-01,Overall Portfolio,575.0,1.875,499.590698
206,2020-04-01,Overall Portfolio,875.0,2.250,627.264573
207,2020-05-01,Overall Portfolio,1275.0,2.250,1011.618216
208,2020-06-01,Overall Portfolio,1675.0,2.250,1507.171894
209,2020-07-01,Overall Portfolio,2075.0,2.250,1959.125229
210,2020-08-01,Overall Portfolio,2475.0,2.250,2357.943719
211,2020-09-01,Overall Portfolio,2875.0,2.250,2856.447969
212,2020-10-01,Overall Portfolio,3675.0,2.620,3616.811898
213,2020-11-01,Overall Portfolio,4475.0,2.990,4640.218352


# EDA timeseries computation

In [143]:
df_orders

,Index,Date,Price,Investment,Ordercost,Depotprovider,Name,ISIN
0,0,2020-02-04,54.2143,50.0,-0.00,flatex,Xtrackers EURO STOXX 50 UCITS ETF 1C,LU0380865021
1,0,2020-02-17,30.9100,50.0,-0.00,traderepublic,iShares BRIC 50 UCITS ETF,IE00B1W57M07
2,0,2020-02-17,27.7100,25.0,-0.00,traderepublic,iShares Core MSCI Emerging Markets IMI UCITS E...,IE00BKM4GZ66
3,0,2020-02-17,6.9200,25.0,-0.00,traderepublic,iShares Edge MSCI USA Value Factor UCITS ETF,IE00BD1F4M44
4,0,2020-02-17,311.3800,25.0,-0.00,traderepublic,iShares Core S&P 500 UCITS ETF (Acc),IE00B5BMR087
...,...,...,...,...,...,...,...,...
157,11,2021-01-05,50.9400,25.0,0.37,comdirect,EUWAX Gold,DE000EWG0LD1
158,11,2021-01-04,33.8600,25.0,-0.00,comdirect,VanEck Vectors Global Real Estate UCITS ETF,NL0009690239
159,11,2021-01-04,116.7800,50.0,-0.00,comdirect,Amundi Nasdaq 100 UCITS ETF EUR (C),LU1681038243
160,11,2021-01-04,92.4400,25.0,-0.00,comdirect,ComStage Commerzbank Commodity ex-Agriculture ...,LU0419741177


# Start implementation

In [144]:
df_orders["Amount"] = df_orders["Investment"]/df_orders["Price"]
group_columns = ["Investment", "Ordercost", "Amount"]
df_orders["Date"] = df_orders["Date"] - pd.offsets.MonthBegin(1)

In [173]:
all_stocks = pd.DataFrame(df_orders["Name"].drop_duplicates()).copy()
all_stocks["key"] = 0
all_dates = pd.DataFrame(df_orders["Date"].drop_duplicates()).copy()
all_dates["key"] = 0
all_combinations = pd.merge(all_dates, all_stocks, on='key').drop("key", axis=1)
#print(all_stocks.count()[0]* all_dates.count()[0])

In [209]:
(1+all_stocks.count()[0])*all_dates.count()[0]

216

In [174]:
df_orders.head()

,Index,Date,Price,Investment,Ordercost,Depotprovider,Name,ISIN,Amount
0,0,2020-02-01,54.2143,50.0,-0.0,flatex,Xtrackers EURO STOXX 50 UCITS ETF 1C,LU0380865021,0.922266
1,0,2020-02-01,30.9100,50.0,-0.0,traderepublic,iShares BRIC 50 UCITS ETF,IE00B1W57M07,1.617599
2,0,2020-02-01,27.7100,25.0,-0.0,traderepublic,iShares Core MSCI Emerging Markets IMI UCITS E...,IE00BKM4GZ66,0.902201
3,0,2020-02-01,6.9200,25.0,-0.0,traderepublic,iShares Edge MSCI USA Value Factor UCITS ETF,IE00BD1F4M44,3.612717
4,0,2020-02-01,311.3800,25.0,-0.0,traderepublic,iShares Core S&P 500 UCITS ETF (Acc),IE00B5BMR087,0.080288


In [175]:
df_start = all_combinations.merge(df_orders[["Date", "Name"]+group_columns], how="left", 
                                                                              left_on=["Date", "Name"],
                                                                              right_on=["Date", "Name"]
                                 ).fillna(0).copy()
prices = df_orders[["Date", "Name", "Price"]].copy()

In [183]:
df_start.sort_values("Date").count()

Date          204
Name          204
Investment    204
Ordercost     204
Amount        204
dtype: int64

In [191]:
df_grouped = df_start.sort_values("Date").groupby("Name").cumsum()
df_grouped_all = df_start.merge(df_grouped, how="left", left_index=True, right_index=True, suffixes=["_init", None])
df_grouped_all = df_grouped_all.drop(["Investment_init", "Ordercost_init", "Amount_init"], axis=1)
df_grouped_all = df_grouped_all.merge(prices, how="left", left_on=["Date", "Name"], right_on=["Date", "Name"], suffixes=[None, "_y"])
df_grouped_all["Value"] = df_grouped_all["Amount"] * df_grouped_all["Price"]
df_grouped_all = df_grouped_all.drop(["Amount", "Price"], axis=1).fillna(0)

In [192]:
df_grouped_all.count()

Date          204
Name          204
Investment    204
Ordercost     204
Value         204
dtype: int64

In [193]:
df_grouped_all[df_grouped_all["Date"] <= all_dates["Date"].iloc[3]].sort_values("Date")#.sort_values("Name")

,Date,Name,Investment,Ordercost,Value
0,2020-02-01,Xtrackers EURO STOXX 50 UCITS ETF 1C,50.0,0.00,50.000000
16,2020-02-01,VanEck Vectors Global Real Estate UCITS ETF,0.0,0.00,0.000000
15,2020-02-01,Amundi Nasdaq 100 UCITS ETF EUR (C),0.0,0.00,0.000000
14,2020-02-01,Vanguard FTSE All-World High Dividend Yield UC...,0.0,0.00,0.000000
13,2020-02-01,Xtrackers Russell 2000 UCITS ETF 1C,0.0,0.00,0.000000
...,...,...,...,...,...
52,2020-05-01,iShares BRIC 50 UCITS ETF,200.0,0.00,197.253178
51,2020-05-01,Xtrackers EURO STOXX 50 UCITS ETF 1C,100.0,0.75,0.000000
66,2020-05-01,Amundi Nasdaq 100 UCITS ETF EUR (C),25.0,0.00,25.000000
58,2020-05-01,iShares S&P SmallCap 600 UCITS ETF,75.0,0.00,74.621778


In [194]:
df_cumsum = df_grouped_all.sort_values("Date").set_index("Date").drop(["Name"], axis=1).groupby("Date").sum()

In [195]:
df_cumsum#.cumsum()

,Investment,Ordercost,Value
Date,,,
2020-02-01,175.0,0.000,175.000000
2020-03-01,575.0,1.875,499.590698
2020-04-01,875.0,2.250,627.264573
2020-05-01,1275.0,2.250,1011.618216
2020-06-01,1675.0,2.250,1507.171894
2020-07-01,2075.0,2.250,1959.125229
2020-08-01,2475.0,2.250,2357.943719
2020-09-01,2875.0,2.250,2856.447969
2020-10-01,3675.0,2.620,3616.811898


In [197]:
6041.69+30.94+2.36

6074.989999999999